In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("C:/Users/DELL/Downloads/IMDB Dataset.csv")


In [6]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
data.shape

(50000, 2)

In [8]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [9]:
# Encode label data
data.replace({'sentiment':{'positive':1, 'negative':0}}, inplace=True)
data.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_2880\2772124872.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment':{'positive':1, 'negative':0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
from sklearn.model_selection import train_test_split

train_data, test_data=train_test_split(data, test_size=0.2, random_state=42)

In [11]:
train_data.shape, test_data.shape

((40000, 2), (10000, 2))

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

# Convert text to sequences and pad them to ensure equal sequence length
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200) 

In [15]:
#Labels
y_train= train_data['sentiment']
y_test= test_data['sentiment']


## Building & Training LSTM(Long Short Term Memory) Model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Build the model

model=Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

C:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# Model Training
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 325s 633ms/step - accuracy: 0.7214 - loss: 0.5331 - val_accuracy: 0.8418 - val_loss: 0.3742
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 296s 592ms/step - accuracy: 0.8595 - loss: 0.3447 - val_accuracy: 0.8554 - val_loss: 0.3453
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 303s 607ms/step - accuracy: 0.8721 - loss: 0.3126 - val_accuracy: 0.8733 - val_loss: 0.3126
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 297s 593ms/step - accuracy: 0.8872 - loss: 0.2784 - val_accuracy: 0.8795 - val_loss: 0.2996
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 12844s 26s/step - accuracy: 0.9065 - loss: 0.2368 - val_accuracy: 0.8742 - val_loss: 0.3150


## Model Evaluation

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 106ms/step - accuracy: 0.8741 - loss: 0.3082


## Building the prediction system

In [27]:
def predict_sentiment(review):
    sequence=tokenizer.texts_to_sequences([review])
    padded_sequence=pad_sequences(sequences, maxlen=200)
    prediction=model.predict(padded_sequence)
    sentiment="positive" if prediction[0][0] >0.5 else "negative"
    return sentiment

In [28]:
new_review="This movie was fantastic. I loved it!"
sentiment=predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')

NameError: name 'sequences' is not defined